<hr style="border:2px solid #808080"> </hr>
<center><h1 style="color:#03122E;"> Álgebra Lineal Numérica IMT2111</h1></center>
<center><h1 style="color:#173F8A;"> Capítulo 2</h3></center>
<center><h1 style="color:#0176DE;"> Prof. Manuel A. Sánchez</h3></center>
<hr style="border:2px solid #808080"> </hr>

In [3]:
import numpy as np
from numpy.linalg import norm
import matplotlib.pyplot as plt
import pandas as pd
import scipy.sparse as sp
from scipy.sparse import csr_matrix, diags, tril, triu
from scipy.sparse.linalg import spsolve
from scipy.sparse.linalg import splu

In [17]:
def ConjugateGradient(A, b, x0=None, TOL=1e-8, MAXIT=100, print_iter=True):
    '''
    Conjugate Gradient Method: Hestenes-Stiefel Version
    
    Solves the linear system Ax = b for x, where A is a symmetric, positive-definite matrix.
    
    Parameters:
    A : numpy.ndarray
        Symmetric, positive-definite matrix.
    b : numpy.ndarray
        Right-hand side vector.
    x0 : numpy.ndarray, optional
        Initial guess for the solution. If None, a zero vector is used.
    TOL : float, optional
        Tolerance for the stopping criterion. Default is 1e-8.
    MAXIT : int, optional
        Maximum number of iterations. Default is 100.
    print_iter : bool, optional
        If True, prints the norm of the residual at each iteration. Default is True.
        
    Returns:
    x : numpy.ndarray
        Approximate solution to the system Ax = b.
    k : int
        Number of iterations performed.
    '''
    if x0 is None:
        x0 = np.zeros(b.size)
        
    k = 0
    r0 = b - A.dot(x0)
    pk = r0.copy()
    x = x0.copy()
    
    for k in range(MAXIT):
        rk = r0.dot(r0)
        Apk = A.dot(pk)
        alphak = rk / pk.dot(Apk)
        x += alphak * pk
        r0 -= alphak * Apk
        if print_iter:
            print(f"Iteration {k}: Residual norm = {np.linalg.norm(r0)}")
        if np.linalg.norm(r0) < TOL:
            break
        betak = r0.dot(r0) / rk
        pk = r0 + betak * pk
        
    return x, k

# Example usage:
# A = np.array([[4, 1], [1, 3]])
# b = np.array([1, 2])
# x0 = np.zeros(2)
# x, num_iters = ConjugateGradient(A, b, x0)
# print("Solution:", x)
# print("Number of iterations:", num_iters)


In [18]:
def PreconditionedConjugateGradient(A, b, M_func, x0=None, TOL=1e-8, MAXIT=100, print_iter=True):
    '''
    Preconditioned Conjugate Gradient Method
    
    Solves the linear system Ax = b for x, where A is a symmetric, positive-definite matrix,
    using a preconditioner function M_func.
    
    Parameters:
    A : numpy.ndarray
        Symmetric, positive-definite matrix.
    b : numpy.ndarray
        Right-hand side vector.
    M_func : function
        Function that applies the preconditioner.
    x0 : numpy.ndarray, optional
        Initial guess for the solution. If None, a zero vector is used.
    TOL : float, optional
        Tolerance for the stopping criterion. Default is 1e-8.
    MAXIT : int, optional
        Maximum number of iterations. Default is 100.
    print_iter : bool, optional
        If True, prints the norm of the residual at each iteration. Default is True.
        
    Returns:
    x : numpy.ndarray
        Approximate solution to the system Ax = b.
    k : int
        Number of iterations performed.
    '''
    if x0 is None:
        x0 = np.zeros(b.size)
    
    k = 0
    r0 = b - A.dot(x0)
    z0 = M_func(r0)
    pk = z0.copy()
    x = x0.copy()
    
    for k in range(MAXIT):
        rk = r0.dot(z0)
        Apk = A.dot(pk)
        alphak = rk / pk.dot(Apk)
        x += alphak * pk
        r0 -= alphak * Apk
        if print_iter:
            print(f"Iteration {k}: Residual norm = {np.linalg.norm(r0)}")
        if np.linalg.norm(r0) < TOL:
            break
        z0 = M_func(r0)
        betak = r0.dot(z0) / rk
        pk = z0 + betak * pk
    
    return x, k

In [19]:
def jacobi_preconditioner(A):
    """
    Jacobi preconditioner function.
    
    Parameters:
    A : numpy.ndarray
        Symmetric, positive-definite matrix.
        
    Returns:
    M_func : function
        Function that applies the Jacobi preconditioner.
    """
    # Extract the diagonal elements of A
    # D_inv = 1.0 / np.diag(A)
    D_inv = 1.0 /  A.diagonal()
    
    def M_func(r):
        return D_inv * r
    
    return M_func

In [29]:
def gauss_seidel_preconditioner(A):
    """
    Gauss-Seidel preconditioner function.
    
    Parameters:
    A : numpy.ndarray
        Symmetric, positive-definite matrix.
        
    Returns:
    M_func : function
        Function that applies the Gauss-Seidel preconditioner.
    """
    L = tril(A)  # Lower triangular part of A including the diagonal

    def M_func(r):
        # Solve Lz = r using forward substitution
        z = spsolve(L, r)
        return z

    return M_func

In [35]:
def gauss_seidel_symmetric_preconditioner(A):
    """
    Gauss-Seidel symmetric preconditioner function for sparse matrices.
    
    Parameters:
    A : scipy.sparse.csr_matrix
        Sparse, symmetric, positive-definite matrix.
        
    Returns:
    M_func : function
        Function that applies the Gauss-Seidel symmetric preconditioner.
    """
    L = tril(A)  # Lower triangular part of A including the diagonal
    U = triu(A)  # Upper triangular part of A including the diagonal
    
    # Perform incomplete Cholesky factorization on the lower triangular part
    L_chol = splu(L)
    
    def M_func(r):
        # Solve Lz = r using forward substitution
        z = L_chol.solve(r)
        # Solve Uz = z using backward substitution
        z = L_chol.solve(z)
        return z

    return M_func

In [42]:
def sor_preconditioner(A, omega):
    """
    Successive Over-Relaxation (SOR) preconditioner function for sparse matrices.
    
    Parameters:
    A : scipy.sparse.csr_matrix
        Sparse, symmetric, positive-definite matrix.
    omega : float
        Relaxation parameter (0 < omega < 2).
        
    Returns:
    M_func : function
        Function that applies the SOR preconditioner.
    """
    D = diags(A.diagonal())  # Diagonal matrix of A
    L = tril(A, k=-1)        # Lower triangular part of A
    U = triu(A, k=1)         # Upper triangular part of A

    # Calculate the inverse of the diagonal matrix plus omega times the lower triangular matrix
    M_inv = splu(D + omega * L)
    
    def M_func(r):
        # Solve (D + omega * L)z = r using forward substitution
        z = M_inv.solve(r)
        return z

    return M_func


In [43]:
def sor_symmetric_preconditioner(A, omega):
    """
    Symmetric Successive Over-Relaxation (SOR) preconditioner function for sparse matrices.
    
    Parameters:
    A : scipy.sparse.csr_matrix
        Sparse, symmetric, positive-definite matrix.
    omega : float
        Relaxation parameter (0 < omega < 2).
        
    Returns:
    M_func : function
        Function that applies the symmetric SOR preconditioner.
    """
    D = diags(A.diagonal())  # Diagonal matrix of A
    L = tril(A, k=-1)        # Lower triangular part of A
    U = triu(A, k=1)         # Upper triangular part of A

    # Calculate the inverse of the symmetric SOR preconditioner
    M_inv = splu(D + omega * L + omega * U)
    
    def M_func(r):
        # Solve (D + omega * L + omega * U)z = r
        z = M_inv.solve(r)
        return z

    return M_func


In [74]:


def incomplete_cholesky_preconditioner(A):
    """
    Incomplete Cholesky preconditioner function for sparse matrices.
    
    Parameters:
    A : scipy.sparse.csr_matrix
        Sparse, symmetric, positive-definite matrix.
        
    Returns:
    M_func : function
        Function that applies the Incomplete Cholesky preconditioner.
    """
    ILU = sparse_ilu(A)
    ILU_solver = spsolve(ILU.T, np.eye(ILU.shape[0]))

    def M_func(r):
        # Solve Ly = r using forward substitution
        y = spsolve(ILU, r)
        # Solve U^Tz = y using backward substitution
        z = ILU_solver @ y
        return z

    return M_func

def sparse_ilu(A, drop_tol=1e-4):
    """
    Incomplete LU factorization for sparse matrices.
    
    Parameters:
    A : scipy.sparse.csr_matrix
        Sparse, symmetric, positive-definite matrix.
    drop_tol : float, optional
        Threshold to drop elements during factorization. Default is 1e-4.
        
    Returns:
    ILU : scipy.sparse.csr_matrix
        Lower triangular matrix from the Incomplete LU factorization.
    """
    ILU = spilu(A, fill_factor=drop_tol)
    return ILU.L
# def incomplete_cholesky_preconditioner(A):
#     """
#     Incomplete Cholesky preconditioner function for sparse matrices.
    
#     Parameters:
#     A : scipy.sparse.csr_matrix
#         Sparse, symmetric, positive-definite matrix.
        
#     Returns:
#     M_func : function
#         Function that applies the Incomplete Cholesky preconditioner.
#     """
#     L = sparse_ichol(A)
#     L_solver = spsolve(L.T, np.eye(L.shape[0]))

#     def M_func(r):
#         # Solve Ly = r using forward substitution
#         y = spsolve(L, r)
#         # Solve L^Tz = y using backward substitution
#         z = L_solver @ y
#         return z

#     return M_func

# def sparse_ichol(A, drop_tol=1e-4):
#     """
#     Incomplete Cholesky factorization for sparse matrices.
    
#     Parameters:
#     A : scipy.sparse.csr_matrix
#         Sparse, symmetric, positive-definite matrix.
#     drop_tol : float, optional
#         Threshold to drop elements during factorization. Default is 1e-4.
        
#     Returns:
#     L : scipy.sparse.csr_matrix
#         Lower triangular matrix from the Incomplete Cholesky factorization.
#     """
#     n = A.shape[0]
#     L = csr_matrix((n, n))

#     for i in range(n):
#         L[i, i] = np.sqrt(A[i, i])
#         for j in range(i+1, n):
#             if A[j, i] != 0:
#                 L[j, i] = A[j, i] / L[i, i]
#                 for k in range(i):
#                     if L[j, k] != 0 and L[i, k] != 0:
#                         A[j, k] -= L[j, i] * L[i, k]
#             if abs(A[j, i]) < drop_tol * L[i, i]:
#                 A[j, i] = 0

#     return L


In [66]:
from scipy.sparse.linalg import spilu, spsolve

def incomplete_lu_preconditioner(A):
    """
    Incomplete LU preconditioner function for sparse matrices.
    
    Parameters:
    A : scipy.sparse.csr_matrix
        Sparse, symmetric, positive-definite matrix.
        
    Returns:
    M_func : function
        Function that applies the Incomplete LU preconditioner.
    """
    ILU = sparse_ilu(A)
    ILU_solver = spsolve(ILU.T, np.eye(ILU.shape[0]))

    def M_func(r):
        # Solve Ly = r using forward substitution
        y = spsolve(ILU, r)
        # Solve U^Tz = y using backward substitution
        z = ILU_solver @ y
        return z

    return M_func

def sparse_ilu(A, fill_factor=10):
    """
    Incomplete LU factorization for sparse matrices.
    
    Parameters:
    A : scipy.sparse.csr_matrix
        Sparse, symmetric, positive-definite matrix.
    fill_factor : int, optional
        Fill factor for the ILU factorization. Default is 10.
        
    Returns:
    ILU : scipy.sparse.csr_matrix
        Lower triangular matrix from the Incomplete LU factorization.
    """
    ILU = spilu(A, fill_factor=fill_factor)
    return ILU.L



In [68]:
import numpy as np
from scipy.sparse import kron
from scipy.sparse.linalg import spilu

def block_diagonal_preconditioner(A):
    """
    Block diagonal preconditioner function for sparse matrices.
    
    Parameters:
    A : scipy.sparse.csr_matrix
        Sparse matrix.
        
    Returns:
    M_func : function
        Function that applies the block diagonal preconditioner.
    """
    # Perform ILU(0) factorization on the block diagonal part of A
    block_size = int(np.sqrt(A.shape[0]))
    blocks = [A[i*block_size:(i+1)*block_size, i*block_size:(i+1)*block_size] for i in range(block_size)]
    block_ILUs = [spilu(B) for B in blocks]

    def M_func(r):
        # Apply the preconditioner block by block
        z = np.concatenate([block_ILUs[i].solve(r[i*block_size:(i+1)*block_size]) for i in range(block_size)])
        return z

    return M_func


In [30]:
# Example usage:
A = np.array([[4, 1], [1, 3]])
b = np.array([1, 2])
M_func = jacobi_preconditioner(A)
x0 = np.zeros(2)
x, num_iters = PreconditionedConjugateGradient(A, b, M_func, x0)
print("Solution:", x)
print("Number of iterations:", num_iters)

Iteration 0: Residual norm = 0.40243495901857923
Iteration 1: Residual norm = 0.0
Solution: [0.09090909 0.63636364]
Number of iterations: 1


In [31]:
def testproblem(num, m):
    if num == 1:
    # Poisson matrix
        a = -1;
        b = a;
        c = 2;
    elif num == 2:
    # Averaging problem
        a = 1.0/9.0;
        b = a;
        c = 5.0/18.0;
    # end 
    C1 = c*np.diag(np.ones(m),0)+a*np.diag(np.ones(m-1),-1)+a*np.diag(np.ones(m-1),1);
    C2 = c*np.diag(np.ones(m),0)+b*np.diag(np.ones(m-1),-1)+b*np.diag(np.ones(m-1),1)
    A = np.kron(np.eye(m),C1) + np.kron(C2, np.eye(m))
    bvector = np.random.rand(m*m)
    return sp.csr_matrix(A), bvector

In [75]:
def printCG(iter, res):
    print(f" *** CG iteration {iter}, residual = {res:1.2e}")   
def TestCG_example(examplenumber, mm=5):
    kappa = np.zeros(mm)
    nn =np.zeros(mm, dtype=np.int64)
    M = [5*2**i for i in range(mm)]
    for m in range(len(M)):
        A, b = testproblem(examplenumber, M[m])
        x, nn[m] = ConjugateGradient(A,b, MAXIT=1000, print_iter=False)
        kappa[m] = np.linalg.cond(A.todense())
    tab = pd.DataFrame({'m':M, 'Matrix dim':np.square(M), 'Condition number':kappa, 'number of CG iterations': nn})
    return tab
def TestPCG_example(examplenumber, mm=5):
    kappa = np.zeros(mm)
    nn =np.zeros(mm, dtype=np.int64)
    M = [5*2**i for i in range(mm)]
    for m in range(len(M)):
        A, b = testproblem(examplenumber, M[m])
        # M_func = jacobi_preconditioner(A)
        # M_func = gauss_seidel_preconditioner(A)
        # M_func = gauss_seidel_symmetric_preconditioner(A)
        # M_func = sor_preconditioner(A,omega=1.27)
        # M_func = sor_symmetric_preconditioner(A,omega=1.65)
        M_func = incomplete_cholesky_preconditioner(A)
        # M_func = incomplete_lu_preconditioner(A)
        # M_func = block_diagonal_preconditioner(A)
        x, nn[m] = PreconditionedConjugateGradient(A,b, M_func, MAXIT=1000, print_iter=False)
        kappa[m] = 1.0%np.linalg.cond(A.todense())
    tab = pd.DataFrame({'m':M, 'Matrix dim':np.square(M), 'Condition number':kappa, 'number of CG iterations': nn})
    return tab

In [33]:
tab1 = TestCG_example(examplenumber=1, mm=5)
tab1 = tab1.style.set_caption('Example, increasing condition number')
tab1

,m,Matrix dim,Condition number,number of CG iterations
0,5,25,13.928203,12
1,10,100,48.374150,33
2,20,400,178.064275,67
3,40,1600,680.617070,134
4,80,6400,2658.406502,271


In [29]:
tab2 = TestCG_example(examplenumber=2, mm=5)
tab2 = tab2.style.set_caption('Example, bounded condition number')
tab2

,m,Matrix dim,Condition number,number of CG iterations
0,5,25,5.510847,12
1,10,100,7.605644,22
2,20,400,8.572340,28
3,40,1600,8.883994,29
4,80,6400,8.970008,30


In [ ]:
tab3 = TestPCG_example(examplenumber=1, mm=5)
tab3 = tab3.style.set_caption('Example, increasing condition number')
tab3

dtype('int64')

In [6]:
from scipy.sparse.linalg import cg

Afree = Aload[fd,:][:,fd]
bfree = bload[fd]
xsol = np.zeros(bload.size, dtype=np.float64)
xsol[fd], exit_code = cg(Afree, bfree, atol=1e-8)

In [7]:
print(exit_code)

0


In [12]:

from ngsolve import *
from ngsolve.webgui import Draw

In [13]:
mesh = Mesh('mesh_elasticity.vol')
Draw(mesh);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.22…

In [14]:
fes = VectorH1(mesh, order=1, dirichlet="fix")
print(xsol.size, fes.ndof)
gfu = GridFunction(fes)
gfu.vec.FV().NumPy()[:] = xsol

6873 6873


In [15]:
Draw(gfu)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.22…

BaseWebGuiScene